In [6]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("shashwatwork/knee-osteoarthritis-dataset-with-severity")

In [7]:
train_dir = os.path.join(path, "train")
val_dir = os.path.join(path, "val")
test_dir = os.path.join(path, "test")

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

batch_size = 32
img_size = (224, 224)
num_classes = 5

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    batch_size=batch_size
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    batch_size=batch_size
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1, 0.1)
])

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(x, y):
    return preprocess_input(x), y

train_dataset = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)
val_dataset = val_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 5778 files belonging to 5 classes.
Found 826 files belonging to 5 classes.
Found 1656 files belonging to 5 classes.


In [9]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

def build_model(hp):
    base_model1 = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model1.trainable = True  # Fine-tune the base

    model1 = models.Sequential()
    model1.add(base_model1)
    model1.add(layers.GlobalAveragePooling2D())

    # Tune number of units in Dense layer
    hp_units_1 = hp.Int('dense_units', min_value=128, max_value=512, step=64)

    # Tune activation function
    activation = hp.Choice('activation', ['relu', 'leaky_relu'])
    if activation == 'leaky_relu':
        model1.add(layers.Dense(hp_units_1))
        model1.add(layers.LeakyReLU())
    else:
        model1.add(layers.Dense(hp_units_1, activation='relu'))

    # Tune dropout rate
    hp_dropout = hp.Float('dropout_rate', 0.3, 0.75, step=0.05)
    model1.add(layers.Dropout(hp_dropout))

    model1.add(layers.Dense(5, activation='softmax'))

    # Tune learning rate
    learning_rate = hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')

    model1.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model1

# Set up tuner
tuner1 = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='kt_dir',
    project_name='resnet_tuning'
)

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Search for best hyperparameters
tuner1.search(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, lr_schedule]
)

# Retrieve and train best model
best_hp = tuner1.get_best_hyperparameters(1)[0]
model1 = tuner1.hypermodel.build(best_hp)

history = model1.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop, lr_schedule]
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
384               |384               |dense_units
leaky_relu        |leaky_relu        |activation
0.55              |0.55              |dropout_rate
0.00088181        |0.00088181        |learning_rate
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


In [ ]:
best_hp1 = tuner1.get_best_hyperparameters(1)[0]
model1 = tuner1.hypermodel.build(best_hp1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history1 = model1.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop, lr_schedule]
)

In [ ]:

loss, accuracy = model1.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

def build_model(hp):
    base_model2 = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model2.trainable = True

    model2 = models.Sequential()
    model2.add(base_model2)
    model2.add(layers.GlobalAveragePooling2D())

    # Tune number of units in Dense layer
    hp_units2 = hp.Int('dense_units', min_value=128, max_value=512, step=64)

    # Tune activation function
    activation = hp.Choice('activation', ['relu', 'leaky_relu'])
    if activation == 'leaky_relu':
        model2.add(layers.Dense(hp_units2))
        model2.add(layers.LeakyReLU())
    else:
        model2.add(layers.Dense(hp_units2, activation='relu'))

    # Tune dropout rate
    hp_dropout = hp.Float('dropout_rate', 0.3, 0.75, step=0.05)
    model2.add(layers.Dropout(hp_dropout))

    model2.add(layers.Dense(5, activation='softmax'))

    # Tune learning rate
    learning_rate = hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')

    model2.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model2

# Set up the tuner
tuner2 = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='kt_densenet',
    project_name='densenet_tuning'
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Start the hyperparameter search
tuner2.search(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, lr_schedule]
)


In [ ]:
# Retrieve and train the best model
best_hp2 = tuner2.get_best_hyperparameters(1)[0]
model2 = tuner2.hypermodel.build(best_hp2)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history2 = model2.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop, lr_schedule]
)

In [ ]:

loss, accuracy = model2.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

def build_model(hp):
    base_model_vgg = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model_vgg.trainable = True

    model_vgg = models.Sequential()
    model_vgg.add(base_model_vgg)
    model_vgg.add(layers.GlobalAveragePooling2D())

    # Tune number of units in Dense layer
    hp_units_vgg = hp.Int('dense_units', min_value=128, max_value=512, step=64)

    # Tune activation function
    activation = hp.Choice('activation', ['relu', 'leaky_relu'])
    if activation == 'leaky_relu':
        model_vgg.add(layers.Dense(hp_units_vgg))
        model_vgg.add(layers.LeakyReLU())
    else:
        model_vgg.add(layers.Dense(hp_units_vgg, activation='relu'))

    # Tune dropout rate
    hp_dropout = hp.Float('dropout_rate', 0.3, 0.75, step=0.05)
    model_vgg.add(layers.Dropout(hp_dropout))

    model_vgg.add(layers.Dense(5, activation='softmax'))

    # Tune learning rate
    learning_rate = hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')

    model_vgg.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model_vgg

# Set up the tuner
tuner_vgg = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='kt_vgg16',
    project_name='vgg16_tuning'
)

# Callbacks
early_stop_vgg = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Run the search
tuner_vgg.search(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, lr_schedule]
)


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Retrieve and train the best model
best_hp_vgg = tuner_vgg.get_best_hyperparameters(1)[0]
model_vgg = tuner_vgg.hypermodel.build(best_hp_vgg)

history_vgg = model_vgg.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop, lr_schedule]
)

In [ ]:

loss, accuracy = model_vgg.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

def build_model(hp):
    base_model_efficientnet = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model_efficientnet.trainable = True

    model_efficientnet = models.Sequential()
    model_efficientnet.add(base_model_efficientnet)
    model_efficientnet.add(layers.GlobalAveragePooling2D())

    # Tune number of units in Dense layer
    hp_units_efficientnet = hp.Int('dense_units', min_value=128, max_value=512, step=64)

    # Tune activation function
    activation = hp.Choice('activation', ['relu', 'leaky_relu'])
    if activation == 'leaky_relu':
        model_efficientnet.add(layers.Dense(hp_units_efficientnet))
        model_efficientnet.add(layers.LeakyReLU())
    else:
        model_efficientnet.add(layers.Dense(hp_units_efficientnet, activation='relu'))

    # Tune dropout rate
    hp_dropout = hp.Float('dropout_rate', 0.3, 0.75, step=0.05)
    model_efficientnet.add(layers.Dropout(hp_dropout))

    model_efficientnet.add(layers.Dense(5, activation='softmax'))

    # Tune learning rate
    learning_rate = hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')

    model_efficientnet.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model_efficientnet

# Set up the tuner
tuner_efficientnet = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='kt_efficientnet',
    project_name='efficientnet_tuning'
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Run the search
tuner_efficientnet.search(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, lr_schedule]
)

# Retrieve and train the best model
best_hp_efficientnet = tuner_efficientnet.get_best_hyperparameters(1)[0]
model_efficientnet = tuner_efficientnet.hypermodel.build(best_hp)

history_efficientnet = model_efficientnet.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop, lr_schedule]
)


In [ ]:

loss, accuracy = model_efficientnet.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:

import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

predictions1 = model1.predict(test_dataset)
predictions2 = model2.predict(test_dataset)
predictions_vgg = model_vgg.predict(test_dataset)
predictions_efficientnet = model_efficientnet.predict(test_dataset)


y_pred1 = np.argmax(predictions1, axis=1)
y_pred2 = np.argmax(predictions2, axis=1)
y_pred_vgg = np.argmax(predictions_vgg, axis=1)
y_pred_efficientnet = np.argmax(predictions_efficientnet, axis=1)


X_test_stacked = np.column_stack((y_pred1, y_pred2, y_pred_vgg, y_pred_efficientnet))
y_test = np.concatenate([y for x, y in test_dataset], axis=0)


estimators = [
    ('model1', model1),
    ('model2', model2),
    ('model_vgg', model_vgg),
    ('model_efficientnet', model_efficientnet)
]

stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)


stacking_model.fit(X_test_stacked, y_test)

stacking_predictions = stacking_model.predict(X_test_stacked)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, stacking_predictions)
print(f"Stacking Model Accuracy on Test Set: {accuracy:.4f}")


In [ ]:
# prompt: Create a graph of validation loss vs training loss

import matplotlib.pyplot as plt

# Assuming 'history' object contains the training history from model.fit
# history = model.fit(...)

# Access training and validation loss values
training_loss = model1.history.history['loss']
validation_loss = model1.history.history['val_loss']

# Create the plot
epochs = range(1, len(training_loss) + 1)
plt.plot(epochs, training_loss, 'b', label='Training Loss')
plt.plot(epochs, validation_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
